<a href="https://colab.research.google.com/github/you0229/kaggle_titanic/blob/master/main_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# libraries
import pandas as pd
from keras import models
from keras import layers
from keras.utils import to_categorical

In [0]:
# データ読み込み
def read_data():
  print('Train data reading...')
  train = pd.read_csv('/content/drive/My Drive/MachineLeaning/kaggle_titanic/train.csv')
  print("Train data is column {}, rows: {}".format(train.shape[0], train.shape[1]))
  print('Test data reading...')
  test = pd.read_csv('/content/drive/My Drive/MachineLeaning/kaggle_titanic/test.csv')
  print("Test data is column {}, rows: {}".format(test.shape[0], test.shape[1]))

  return train, test

In [0]:
# データ整理
def create_data(train, test):
  # train と test を合わせて前処理
  print('Combine train and test')
  train['is_train'] = 1
  test['is_train'] = 0
  train_test = pd.concat([train.drop('Survived', axis=1),test], axis=0)
  display(train_test.info())

  # Sex
  print('Modifying Sex column')
  train_test = train_test.replace('female', 0).replace('male', 1)

  # Ticket
  print('Modifying Ticket column')
  ticket_to_num = {'A':0, 'P':1, 'S':2, '1':3, '3':4, '2':5, 'C':6, '7':7, 'W':8, '4':9, 'F':9, 'L':10, '9':11,'6':12, '5':13, '8':14}
  train_test['Ticket_Left'] = train_test['Ticket'].apply(lambda x: str(x)[0])
  train_test['Ticket_Left'] = train_test['Ticket_Left'].replace(ticket_to_num)
  train_test = train_test.drop('Ticket', axis=1)

  # Embarked
  print('Modifying Embarked column')
  embarked_to_num = {'S':0, 'C':1, 'Q':2}
  train_test['Embarked'] = train_test['Embarked'].replace(embarked_to_num)
  train_test['Embarked'] = train_test['Embarked'].fillna(train_test['Embarked'].mean())
  train_test = train_test.drop('Embarked', axis=1)

  # Cabin
  # 204/891 したデータが入っていないため削除
  print('Modifying Cabin column')
  train_test = train_test.drop('Cabin', axis=1)

  # Age
  # print('Modifying Age column')
  # train_test = train_test.dropna(subset=['Age'])

  # Name
  # 関係なさそうだから削除
  print('Modifying Name column')
  train_test = train_test.drop('Name', axis=1)

  # Fare
  # print('Modifying Fare column')
  # train_test = train_test.dropna(subset=['Fare'])

  # train test
  print('Split train and test')
  train_result = train_test.loc[train['is_train'] == 1]
  test_result = train_test.loc[train['is_train'] == 0]
  train_result = train.drop('is_train', axis=1)
  test_result = test.drop('is_train', axis=1)

  # add Survived
  train_result['Survived'] = train['Survived']

  # Delete Age and Fare
  train_result = train_result.dropna(subset=['Age', 'Fare'])

  return train_result, test_result

In [0]:
# モデル作成
def create_model():
  network = models.Sequential()
  network.add(layers.Dence(512, activation='rule', input_shape=(1045 * 8,)))
  network.add(layers.Dence(10,activation='softmax'))
  network.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

  return network

In [0]:
# ラベルの処理
def data_to_categorical(train):
  return to_categorical(train_labels)

In [0]:
""" data test """
display(train.shape)
display(test.shape)
""" end """

(1045, 8)

(331, 8)

' end '

In [0]:
# 学習

In [0]:
# 評価

In [17]:
train, test = read_data()
display(train.head(3))
display(test.head(3))

Train data reading...
Train data is column 891, rows: 12
Test data reading...
Test data is column 418, rows: 11


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


In [18]:
train, test = create_data(train, test)
display(train.head(3))
display(test.head(3))

Combine train and test
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null object
Age            1046 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1308 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
is_train       1309 non-null int64
dtypes: float64(2), int64(5), object(5)
memory usage: 132.9+ KB


None

Modifying Sex column
Modifying Ticket column
Modifying Embarked column
Modifying Cabin column
Modifying Name column
Split train and test


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


In [0]:
model = create_model()
